In [7]:
# Imports
import os
import gzip
from rdkit import Chem
from rdkit.Chem import AllChem
from meeko import MoleculePreparation, PDBQTWriterLegacy

In [2]:
# Paths
folder = "1a4g_na"
ligands_file = os.path.join(folder, "na-ligands.sdf.gz")
decoys_file = os.path.join(folder, "na-decoys.sdf.gz")

output_ligands_dir = os.path.join(folder, "prepared_ligands")
output_decoys_dir = os.path.join(folder, "prepared_decoys")

os.makedirs(output_ligands_dir, exist_ok=True)
os.makedirs(output_decoys_dir, exist_ok=True)


In [14]:
def load_molecules_from_gz_sdf(path):
    with gzip.open(path, 'rb') as f:
        suppl = Chem.ForwardSDMolSupplier(f)
        molecules = [mol for mol in suppl if mol is not None]
    return molecules

def prepare_and_save_pdbqt(mol, output_path):
    # Add hydrogens and embed 3D if no coordinates
    mol = Chem.AddHs(mol)
    if not mol.GetConformer().Is3D():
        AllChem.EmbedMolecule(mol)
        AllChem.UFFOptimizeMolecule(mol)
    
    # Prepare molecule for docking with Meeko
    prep = MoleculePreparation()
    molecule_setups = prep.prepare(mol)

    # take first setup
    molecule_setup = molecule_setups[0]

    # write pdbqt file
    writer = PDBQTWriterLegacy()
    pdbqt_string = writer.write_string(molecule_setup)
    print(pdbqt_string)

    # Save to file
    with open(output_path, "w") as f:
        f.write(pdbqt_string)

In [16]:
# Load molecules
print("Loading ligands...")
ligands = load_molecules_from_gz_sdf(ligands_file)
print(f"Loaded {len(ligands)} ligands.")

print("Loading decoys...")
decoys = load_molecules_from_gz_sdf(decoys_file)
print(f"Loaded {len(decoys)} decoys.")


Loading ligands...
Loaded 40 ligands.
Loading decoys...
Loaded 1200 decoys.
